# Sudoku Puzzle Solver

Some test puzzles of varying difficulties are encoded in a 2 dimensional matrix below.


In [1]:
# Some puzzles for testing
diabolical = [
    [0, 0, 8, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 6, 0, 0, 4, 9, 0],
    [5, 0, 0, 0, 0, 0, 0, 7, 0],
    [0, 7, 0, 0, 4, 0, 0, 0, 0],
    [0, 5, 0, 2, 0, 6, 0, 0, 0],
    [8, 0, 0, 7, 9, 0, 0, 1, 0],
    [0, 6, 3, 0, 0, 0, 0, 0, 1],
    [0, 0, 5, 0, 7, 3, 0, 0, 0],
    [0, 0, 0, 9, 0, 0, 7, 5, 0]
]

hard = [
    [0, 0, 4, 5, 0, 7, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 9, 8],
    [0, 0, 2, 0, 6, 0, 0, 3, 0],
    [7, 0, 0, 1, 5, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 9, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 5, 6],
    [0, 8, 6, 0, 4, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 1, 7, 0],
    [0, 3, 0, 0, 0, 1, 0, 0, 0]
]

moderate = [
    [0, 0, 7, 5, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 9, 8, 0, 0],
    [0, 6, 0, 0, 1, 0, 4, 3, 0],
    [8, 0, 5, 0, 0, 2, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 2, 0, 0],
    [0, 1, 0, 7, 0, 0, 0, 0, 9],
    [0, 0, 3, 0, 0, 8, 0, 0, 4],
    [0, 4, 0, 9, 0, 0, 3, 0, 0],
    [9, 0, 0, 0, 0, 6, 0, 2, 0]
]

easy = [
    [7, 4, 3, 8, 0, 0, 0, 0, 0],
    [0, 0, 0, 4, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 9, 6, 0, 0, 0],
    [0, 5, 0, 0, 8, 0, 0, 6, 0],
    [8, 0, 4, 7, 0, 9, 3, 0, 0],
    [0, 0, 0, 0, 0, 5, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 9],
    [9, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 6, 0, 0, 0, 0, 7, 8, 2]
]

kids = [
    [8, 9, 0, 4, 0, 0, 0, 5, 6],
    [1, 4, 0, 3, 5, 0, 0, 9, 0],
    [0, 0, 0, 0, 0, 0, 8, 0, 0],
    [9, 0, 0, 0, 0, 0, 2, 0, 0],
    [0, 8, 0, 9, 6, 5, 0, 4, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 5],
    [0, 0, 8, 0, 0, 0, 0, 0, 0],
    [0, 3, 0, 0, 2, 1, 0, 7, 8],
    [4, 2, 0, 0, 0, 6, 0, 1, 3]
]

## Checking puzzles

Defining a couple of functions to check our puzzle grid. For now, just a basic test that the puzzle is the expected size.

*TODO:* Should check that the puzzle is internally consistent and hasn't broken the rules.


In [2]:
def check_puzzle_is_ok(puzzle):
    '''Roughly check chosen puzzle "shape" by counting elements'''
    flat_list = [item for sublist in puzzle for item in sublist]
    if len(flat_list) != 81:
        raise ValueError("Expecting 9x9 matrix to have 81 elements")
    return True


We'll also check that a puzzle has been validly solved:

1. Has a value for every cell
2. That value is not repeated in its row, or column
3. That value is not repeated in its cage


In [3]:
def extract_cage_from_matrix(puzzle, cage):
    '''Given a matrix "p", extract the elements that can be found in the 3x3 cage c.'''
    '''Cages are numbered 0..8 with top left being 0 and bottom right being 8.'''
    r = []

    top_x = (cage // 3) * 3
    top_y = (cage % 3) * 3
    for x in range(top_x, top_x+3):
        for y in range(top_y, top_y+3):
            r.append(puzzle[x][y])
    return r
    
def check_solution_is_valid(puzzle, is_complete=True):
    ''' Check that each row, column, and cage has digits 1..9 with no repeats
        If is_complete is True, caller is claiming that the puzzle has been completed,
        so function will reject an incomplete solution. Otherwise will just check 
        that values provided so far don't appear to break the rules yet.'''

    complete_list = set(range(1, 10))
    
    # Check each row
    for row in range(9):
        check_list = puzzle[row]
        if is_complete:
            if set(check_list) != complete_list:
                raise ValueError(f"Row {row} broke the rules: {check_list} {is_complete}")
        else:
            check_list = [x for x in check_list if x != 0]
            if len(check_list) != len(set(check_list)):
                raise ValueError(f"Row {row} contains duplicates: {check_list}")
            
    # Check each column
    for col in range(9):
        check_list = [row[col] for row in puzzle]
        if is_complete:
            if set(check_list) != complete_list:
                raise ValueError(f"Column {col} broke the rules: {check_list}")
        else:
            check_list = [x for x in check_list if x != 0]
            if len(check_list) != len(set(check_list)):
                raise ValueError(f"Column {col} contains duplicates: {check_list}")
            
            
    # Check each cage
    for c in range(9):
        check_list = extract_cage_from_matrix(puzzle, c)
        if is_complete:
            if set(check_list) != complete_list:
                raise ValueError(f"Cage {c} broke the rules: {check_list}")
        else:
            check_list = [x for x in check_list if x != 0]
            if len(check_list) != len(set(check_list)):
                raise ValueError(f"Cage {c} contains duplucates: {check_list}")

        
    return True


## The possibility matrix

We build a "possibility matrix" which is a 9x9 matrix, with each cell initially containing the set of integers from 1 to 9 (inclusive). The idea then is to check the puzzle and update the possibility matrix by excluding any integers according to the Sudoku rules:

1. No integer can be repeated in a row
2. No integer can be repeated in a column
3. No integer can be repeated with a "cage", which is the 3x3 box marked by heavy lines in the puzzle

The idea is that once we exclude the impossibilities what should be left are some possible answers.

### Building the matrix

The matrix is built as a list of lists of integer sets. 

In [4]:
def build_possibility_matrix():
    '''Build a 9x9 matrix with each cell containing the set of digits 1..9'''
    ret = [[] for x in range(9)]
    for x in range(9):
        ret[x] = [[] for y in range(9)]
        for y in range(9):
            ret[x][y] = set(range(1,10))
    return ret


### Excluding possibilities based on a single value

We define a function that takes a `value`, and its position (0-based) in the puzzle `row`,`col`, and the current `possibility_matrix`. We then exclude the `value` from all the places in the matrix that it is no longer allowed to appear because it's "locked in" to `row`,`col`.


In [5]:
def exclude_possibility(value, row, col, possibility_matrix):
    '''Exclude value "value" found at row,col from matrix "possibility_matrix"'''
    #print(f"Excluding {value} at {row+1},{col+1}")
    
    # First, value cannot happen anywhere in row "row"
    for y in range(9):
        if value in possibility_matrix[row][y]:
            possibility_matrix[row][y].remove(value)
        
    # Next, value v cannot happen anywhere in column "col"
    for x in range(9):
        if value in possibility_matrix[x][col]:
            possibility_matrix[x][col].remove(value)
            
    # Finally, value v cannot happen in the 3x3 cell that contains row,col
    cell_x = (row // 3)
    cell_y = (col // 3)
    for x in range(cell_x * 3, (cell_x+1)*3):
        for y in range(cell_y*3, (cell_y+1)*3):
            if value in possibility_matrix[x][y]:
                possibility_matrix[x][y].remove(value)
    
    # We now lock in "v" as only possibility at row,col in q
    possibility_matrix[row][col] = set([value])
    return



### Updating the possibility matrix

We walk through the entire puzzle `p`, and for each solved cell, we update the possibility matrix `q` to remove the solved cell value from the unsolved cells where we know it cannot appear.

The function will return the number of solved cells *in total* which is just a way of keepimg track of how we're doing.


In [6]:
def update_possibilities(puzzle, possibility_matrix):
    '''Pass through puzzle and exclude possibilities from possibility_matrix based on what is already solved'''
    solved_cells = 0
    for x in range(9):
        for y in range(9):
            if puzzle[x][y] != 0:
                solved_cells += 1
                exclude_possibility(puzzle[x][y], x, y, possibility_matrix)
    return solved_cells


# Solution Strategy

So far, have only a single solution strategy, which is to maintain a matrix of *possible values* in `q`. We begin be excluding possible values based on what we know already in the puzzle `p`. We can then check to see if that's collapsed any of the list of possible values to just one remaining possibility. When that happens, we update our puzzle `p` with the solved value.

Eventually (or rather quickly, particularly on harder Sudoku puzzles) this strategy runs out of options and there are no cells left in `q` with a single remaining possible value. At this point we have to throw over to the human.

## Solving using remaining possibilities

This function checks the possibility matrix `q`, looking for cells that have only a single possible value remaining for a cell. When we find one, we update puzzle `p` with that value.

Very often, we'll find that the possibility matrix has only one value because we updated the matrix with our current puzzle `p`. That's OK -- we'll just check that the values are the same. We'll also check that we haven't ended up in a state where a cell has *no possible values* -- clearly that would mean we've made a programming mistake.


In [18]:
def update_puzzle_using_possibilities(puzzle, possibility_matrix):
    ''' Check possibility_matrix, to see if any possibilities have collapsed to a single
        value, and if so, update puzzle'''
    num_solved = 0
    for x in range(9):
        for y in range(9):
            if len(possibility_matrix[x][y]) == 1:
                # Only 1 possibility left! Stash it in "n" but without removing from set
                n = next(iter(possibility_matrix[x][y]))

                # We either have never solved this cell (==0) or we solved it already, but
                # make sure we're keeping our matrix consistent
                if puzzle[x][y] == 0:
                    puzzle[x][y] = n
                    print(f"Resolved {x},{y} as {n}")
                    num_solved += 1

                elif puzzle[x][y] != n:
                    raise ValueError(f"Logic error: Expected to find {puzzle[x][y]} at {x},{y} but resolved to {n} instead")

            elif len(possibility_matrix[x][y]) < 1:
                raise ValueError(f"Logic error: Excluded all possibilities at {x},{y}")

    return num_solved


## Displaying the puzzle grid

Occassionally we'll want to take a look at the state of the puzzle so far. We'll print the matrix and show

1. Solved cells
2. Cells with 2 possibilities



In [8]:
from IPython.display import HTML, display

display(HTML('''
<style type="text/css">
td {
    width: 40px;
    height: 40px;
    border: 1px solid #000 !important; 
    text-align: center !important;
}

td:nth-of-type(3n) {    
    border-right: 3px solid red !important;
}

tr:nth-of-type(3n) td {    
    border-bottom: 3px solid red !important;
}

table {
    border: 3px solid red !important;
}
</style>
'''))

def print_puzzle(puzzle, use_possibility_matrix=[]):
    ''' Display a version of the puzzle matrix. If passed a possibility matrix in `use_possibility_matrix,
        then will also show possible values in any cell where there are just 2 possible values remaining. '''
    data = []
    for x in range(9):
        row_to_show = []
        for y in range(9):
            if puzzle[x][y] != 0:
                row_to_show.append(puzzle[x][y])
            elif len(use_possibility_matrix) > 0 and len(use_possibility_matrix[x][y]) <= 2:
                row_to_show.append(use_possibility_matrix[x][y])
            else:
                row_to_show.append(' ')
        data.append(row_to_show)
    
    display(HTML(
       '<table><tr>{}</tr></table>'.format(
           '</tr><tr>'.join(
               '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
           )
    ))
        

# Solution Strategies

So let's give it a go. There are some standard puzzles defined at the top of the page: `diabolical`, `hard`, `moderate`, `easy`, and `kids`. Assign one of these to `p`, and do a quick check the data has been entered consistently.

* **kids**: Starts with 31 solved cells, can be solved using the possibility matrix alone in 11 rounds.
* **easy**: Starts with 24 solved cells, after 13 rounds has 59 solved cells before giving up.
* **moderate**: Starts with 26 solved cells, solves a single new cell, then gives up after the first round.
* **hard**: Starts with 22 solved cells, also solves a single new cell then stops.
* **diabolical**: Starts with 26 solved cells, solves a single new cell, then stops.


In [19]:
import copy
p = copy.deepcopy(kids)
check_puzzle_is_ok(p)
print_puzzle(p)

8,9,,4,,,,5,6
1,4,,3,5,,,9,
,,,,,,8,,
9,,,,,,2,,
,8,,9,6,5,,4,
,,1,,,,,,5
,,8,,,,,,
,3,,,2,1,,7,8
4,2,,,,6,,1,3


## Strategy 1: Possibility Matrix

One of the tools we built above wa the "possibility matrix." We'll build our possibility matrix in `q`, and then update it to exclude values already solved in the initial puzzle.


In [20]:
# Build a "possibility matrix" in q
q = build_possibility_matrix()
n = update_possibilities(p, q)
print(f"Solved {n} cells so far")

Solved 31 cells so far


Having updated the possibility matrix `q` there may now be some cells which only have a single remaining possible value. If so, we will copy those to `p`, and advise the human how many cells were solved this way.

In [21]:
num_solved = update_puzzle_using_possibilities(p, q)
print(f"Solved {num_solved} cells")
print_puzzle(p)

Resolved 1,6 as 7
Resolved 7,3 as 5
Solved 2 cells


8,9,,4,,,,5,6
1,4,,3,5,,7,9,
,,,,,,8,,
9,,,,,,2,,
,8,,9,6,5,,4,
,,1,,,,,,5
,,8,,,,,,
,3,,5,2,1,,7,8
4,2,,,,6,,1,3


Assuming we solved at least one cell this way let's keep going with this strategy until either
1. The puzzle is solved, or
2. We're not solving any more cells this way.

We'll do that using the function `solve_using_possibilities` which will return `True` if the puzzle is solved or `False` if it is forced to give up.


In [22]:
def solve_using_possibilities(puzzle, possibility_matrix):
    ''' Repeatedly update the puzzle matrix wherever the possibility_matrix indicates there is only one
        possible value; then re-calculate the possible values using the updated puzzle.
        Returns when *either* the puzzle is solved (returns True); 
        OR we stop making progress (returns False).
    '''
    
    rounds = 0
    num_just_solved = 0   # How many cells were solved *this* round?
    num_total_solved = 0  # How many cells have been solved in *total*?
    while True:
        rounds += 1
        num_total_solved = update_possibilities(puzzle, possibility_matrix)
        num_just_solved  = update_puzzle_using_possibilities(puzzle, possibility_matrix)
        num_total_solved += num_just_solved
        print(f"Solved {num_just_solved} cells this round, now {num_total_solved} total solved cells")
        
        if num_total_solved == 81 or num_just_solved == 0:
            break

    # Check for solution (or not)
    if num_total_solved == 81:
        print(f"\nHuzzah! Puzzle has been solved in {rounds} rounds!\n")
        return True
    else:
        print(f"\nUh oh. Run out of options after {rounds} rounds...\n")
        return False

is_solved = solve_using_possibilities(p, q)
print_puzzle(p, use_possibility_matrix=q)

Resolved 1,8 as 2
Resolved 6,3 as 7
Resolved 7,0 as 6
Solved 3 cells this round, now 36 total solved cells
Resolved 1,2 as 6
Resolved 1,5 as 8
Resolved 2,7 as 3
Resolved 6,0 as 5
Resolved 7,2 as 9
Resolved 8,3 as 8
Solved 6 cells this round, now 42 total solved cells
Resolved 0,6 as 1
Resolved 3,3 as 1
Resolved 5,3 as 2
Resolved 6,1 as 1
Resolved 7,6 as 4
Resolved 8,2 as 7
Resolved 8,4 as 9
Solved 7 cells this round, now 49 total solved cells
Resolved 0,4 as 7
Resolved 2,3 as 6
Resolved 2,8 as 4
Resolved 3,8 as 7
Resolved 4,6 as 3
Resolved 6,8 as 9
Resolved 8,6 as 5
Solved 7 cells this round, now 56 total solved cells
Resolved 0,5 as 2
Resolved 2,4 as 1
Resolved 4,2 as 2
Resolved 4,8 as 1
Resolved 6,6 as 6
Solved 5 cells this round, now 61 total solved cells
Resolved 0,2 as 3
Resolved 2,2 as 5
Resolved 2,5 as 9
Resolved 4,0 as 7
Resolved 5,6 as 9
Resolved 6,7 as 2
Solved 6 cells this round, now 67 total solved cells
Resolved 2,0 as 2
Resolved 2,1 as 7
Resolved 3,2 as 4
Resolved 5,0 as 

8,9,3,4,7,2,1,5,6
1,4,6,3,5,8,7,9,2
2,7,5,6,1,9,8,3,4
9,5,4,1,8,3,2,6,7
7,8,2,9,6,5,3,4,1
3,6,1,2,4,7,9,8,5
5,1,8,7,3,4,6,2,9
6,3,9,5,2,1,4,7,8
4,2,7,8,9,6,5,1,3



Check that the puzzle is correctly solved, or at least is still valid.


In [28]:
def check_puzzle_state(is_solved, puzzle):
    if is_solved:
        if check_solution_is_valid(puzzle, is_complete=True):
            print("Puzzle is solved, and solution checks out. Done.")
        else:
            raise ValueError("Bug detected -- we claim to have solved a puzzle but solution is not valid")

    elif check_solution_is_valid(puzzle, is_complete=False):
        print("Puzzle isn't solved yet, but work in progress is valid.")

    else:
        raise ValueError("Bug detected -- puzzle is no longer in a valid state")
        
    return

check_puzzle_state(is_solved, p)

Puzzle isn't solved yet, but work in progress is valid.


## Strategy 2: Find the digits that only appear once

Assuming that the puzzle hasn't yet been solved (only the `kids` puzzle is solved solely using the above strategy), we'll need another strategy for making more progress.

The plan is to go row by row, then column by column, and finally cage by cage. For each of those sets we'll take a look at the remaining possibilities in that set (i.e. that row, or column, or cage). If a particular digit appears only once in that set, then that digit *must* be written in to the only cell that it appears in.

So first, let's go row by row.

In [29]:
from collections import Counter

def update_puzzle_using_single_apperance_digits(puzzle, possibility_matrix):
    ''' Scan the possibility_matrix for digits that only appear once in their row, column, or cage.
        Since every digit must appear exactly once in a row, column or cage, such digits can be written
        into the puzzle at that position. 
        Returns the number of cells we solved this way.
    '''
    num_solved = 0
    q = possibility_matrix
    p = puzzle
    
    # Row by row search
    for x in range(9):
        check_list = q[x]
        possibles = [i for sublist in [c for c in check_list if len(c) > 1] for i in sublist]
        cnt = Counter(possibles)
        singles = set([i for i in cnt if cnt[i] == 1])  # list of digits that have only one possible position

        for y in range(9):
            if len(q[x][y]) > 1:
                v = singles & q[x][y]
                if v:
                    p[x][y] = v.pop()
                    num_solved += 1
                    
    # Column by column then
    for y in range(9):
        check_list = [row[y] for row in q]
        possibles = [i for sublist in [c for c in check_list if len(c) > 1] for i in sublist]
        cnt = Counter(possibles)
        singles = set([i for i in cnt if cnt[i] == 1])
        
        for x in range(9):
            if len(q[x][y]) > 1:
                v = singles & q[x][y]
                if v:
                    p[x][y] = v.pop()
                    num_solved += 1
                    
    # Cage by cage next
    for cage in range(9):
        check_list = extract_cage_from_matrix(q, cage)
        possibles = [i for sublist in [c for c in check_list if len(c) > 1] for i in sublist]
        cnt = Counter(possibles)
        singles = set([i for i in cnt if cnt[i] == 1])
        
        top_x = (cage // 3) * 3
        top_y = (cage % 3) * 3
        for x in range(top_x, top_x+3):
            for y in range(top_y, top_y+3):
                v = singles & q[x][y]
                if v:
                    p[x][y] = v.pop()
                    num_solved += 1
        
    return num_solved

num_solved = update_puzzle_using_single_apperance_digits(p, q)
print(f"Solved {num_solved} more cells")

Solved 10 more cells


Hope we added something this way. Worth trying another round of updates.

In [30]:
is_solved = solve_using_possibilities(p, q)
check_puzzle_state(is_solved, p)
print_puzzle(p, use_possibility_matrix=q)

Solved 0 cells this round, now 31 total solved cells

Uh oh. Run out of options after 1 rounds...

Puzzle isn't solved yet, but work in progress is valid.


,,8,,,,,,
1,"{2, 3}",7,6,,"{2, 8}",4,9,
5,,,,,,,7,
,7,,,4,"{1, 8}",,,
,5,,2,,6,,,7
8,,,7,9,5,,1,
7,6,3,,,,,,1
,,5,,7,3,,,
"{2, 4}",,,9,6,,7,5,3


# Results

So far, we have:

* **kids:** Can be solved with strategy 1 alone.
* **easy:** Can be solved with strategy 1 and 2 repeated a few times.
* **moderate:** Gets stuck at 30 solved cells, having tried alternating strategy 1 & 2.
* **hard:** Has the same result as *moderate* (only gets to 26 cells).
* **diabolical:** Get's a little further, solving 64 cells by alternating strategies 1 & 2 several times over. But eventually, it gets stuck.

We've still got some algorithmic options to try later:

1. Backtracking -- basically, brute-force the bastard by trying every possible answer
2. Trying out a [stochastic or consraint-satisfaction](https://en.wikipedia.org/wiki/Sudoku_solving_algorithms) based approach
3. [Knuth's Algorithm X](https://en.wikipedia.org/wiki/Knuth%27s_Algorithm_X) -- whatever that is. Dancing Links? Sounds fun.
4. Some deductive reasoning techniques are explained in this article on [Sudoku techniques](https://www.conceptispuzzles.com/index.aspx?uri=puzzle/sudoku/techniques).